In [1]:
from morph_analysis import *
import os, sys
import pickle
import re
from bs4 import BeautifulSoup
import numpy as np
from hangul_utils import split_syllables, join_jamos
import collections

def dump(data, name):
    filehandler = open(name, "wb")
    pickle.dump(data, filehandler)
    filehandler.close()
def load(name):
    filehandler = open(name, "rb")
    return pickle.load(filehandler)

morph = Morph()

## Load

In [2]:
freq_Tag = load('storage/freq_Tag')
#freq_TagWord = load('storage/freq_TagWord') # output probability를 구하기 위해
#freq_Word = load('storage/freq_Word')
#freq_pTagcTag = load('storage/freq_pTagcTag') # transition probability를 구하기 위해

## From morphological analysis (sample)

In [3]:
query = '우선'
result0 = [[['우', 0, 1, 'ENT', 'NNG NounV 우/NNG', 'XPN NounV 우/XPN', 'XSN NounV 우/XSN'],
  ['서', 2, 3, 'ENT', 'VV VERB-REG3 서/VV'],
  ['ㄴ', 4, 4, 'FUN', 'E VA-1-0 ㄴ/ETM']],
 [['우', 0, 1, 'ENT', 'NNG NounV 우/NNG', 'XPN NounV 우/XPN'],
  ['선', 2, 4, 'ENT', 'NNG NounC 선/NNG']],
 [['우선', 0, 4, 'ENT', 'NNG NounC 우선/NNG']]]

In [4]:
query = '신고전주의정신을' 
morph_result = [[['신', 0, 2, 'ENT', 'NNG NounC 신/NNG', 'XPN NounC 신/XPN'],
  ['고', 3, 4, 'ENT', 'NNG NounV 고/NNG'],
  ['전', 5, 7, 'ENT', 'NNG NounC 전/NNG'],
  ['주의', 8, 11, 'ENT', 'NNG NounV 주의/NNG'],
  ['정', 12, 14, 'ENT', 'XSN NounC 정/XSN'],
  ['신', 15, 17, 'ENT', 'NNG NounC 신/NNG', 'XSN NounC 신/XSN'],
  ['을', 18, 20, 'FUN', 'E VA-1-3 을/ETM', 'J N-8-3 을/JKO']],
 [['신', 0, 2, 'ENT', 'NNG NounC 신/NNG', 'XPN NounC 신/XPN'],
  ['고', 3, 4, 'ENT', 'NNG NounV 고/NNG'],
  ['전', 5, 7, 'ENT', 'NNG NounC 전/NNG'],
  ['주의', 8, 11, 'ENT', 'NNG NounV 주의/NNG'],
  ['정신', 12, 17, 'ENT', 'NNG NounC 정신/NNG'],
  ['을', 18, 20, 'FUN', 'J N-8-3 을/JKO']],
 [['신', 0, 2, 'ENT', 'NNG NounC 신/NNG', 'XPN NounC 신/XPN'],
  ['고전', 3, 7, 'ENT', 'NNG NounC 고전/NNG'],
  ['주의', 8, 11, 'ENT', 'NNG NounV 주의/NNG'],
  ['정신', 12, 17, 'ENT', 'NNG NounC 정신/NNG'],
  ['을', 18, 20, 'FUN', 'J N-8-3 을/JKO']],
 [['신', 0, 2, 'ENT', 'NNG NounC 신/NNG', 'XPN NounC 신/XPN'],
  ['고전주의', 3, 11, 'ENT', 'NNG NounV 고전주의/NNG'],
  ['정신', 12, 17, 'ENT', 'NNG NounC 정신/NNG'],
  ['을', 18, 20, 'FUN', 'J N-8-3 을/JKO']],
 [['신고', 0, 4, 'ENT', 'NNG NounV 신고/NNG'],
  ['전', 5, 7, 'ENT', 'NNG NounC 전/NNG'],
  ['주의', 8, 11, 'ENT', 'NNG NounV 주의/NNG'],
  ['정', 12, 14, 'ENT', 'XSN NounC 정/XSN'],
  ['신', 15, 17, 'ENT', 'NNG NounC 신/NNG', 'XSN NounC 신/XSN'],
  ['을', 18, 20, 'FUN', 'E VA-1-3 을/ETM', 'J N-8-3 을/JKO']],
 [['신고', 0, 4, 'ENT', 'NNG NounV 신고/NNG'],
  ['전', 5, 7, 'ENT', 'NNG NounC 전/NNG'],
  ['주의', 8, 11, 'ENT', 'NNG NounV 주의/NNG'],
  ['정신', 12, 17, 'ENT', 'NNG NounC 정신/NNG'],
  ['을', 18, 20, 'FUN', 'J N-8-3 을/JKO']]]

In [5]:
query = '본받자'
result3 = [[['본', 0, 2, 'ENT', 'NNG NounC 본/NNG'],
  ['받', 3, 5, 'ENT', 'VV VERB-REG 받/VV'],
  ['자', 6, 7, 'FUN', 'E V-0-0 자/E']],
 [['본받', 0, 5, 'ENT', 'VV VERB-REG 본받/VV'], ['자', 6, 7, 'FUN', 'E V-0-0 자/E']]]

In [6]:
a = [[['좁혀', 0, 4, 'ENT', 'VV VERB-REG4 좁히/VV'],
  ['ㅆ다', 5, 7, 'FUN', 'E V-5-0 었/EP+다/EF']]]

In [7]:
# 형태소 조합이 20개인데, 이들을 모두 viterbi를 돌려 확률값을 구하고
# 20개 중에 가장 높은 확률을 가지는 조합을 best로 선택한다.
print(len(morph_result))

6


## Change morph-result to new format for POS (Preprocessing)

In [8]:
a = ['신', 15, 17, 'ENT', 'NNG NounC 신/NNG', 'XSN NounC 신/XSN']
a[-1].split()[-1].split('/')[0]

'신'

In [9]:
# sample_text = '가까웠지만'
# jamo = split_syllables(sample_text)
# print(jamo)
# restored_text = join_jamos(jamo)
# print(restored_text)

In [10]:
'J N-8-3 을/JKO'.split()[-1].split('+')

['을/JKO']

In [11]:
'E V-5-0 었/EP+다/EF'.split()[-1].split('+')

['었/EP', '다/EF']

In [12]:
def orgin_morph_idx(morph_result, idx):
    new_morph_result = [[]] * len(morph_result)
    
    for i, case in enumerate(morph_result):
        str_idx = idx
        temp = []
        for entry in case:
            for pair in entry[-1].split()[-1].split('+'):
                org_morph = pair.split('/')[0]
                len_syllables = len(split_syllables(org_morph))
                temp.append([org_morph, str_idx, str_idx+len_syllables-1])
                #print([org_morph, str_idx, str_idx+len_syllables-1])
                str_idx += len_syllables
        new_morph_result[i] = temp
        #print('===')
    return new_morph_result 

In [13]:
def check_equal(cur_entry, list):
    for entry in list:
        if cur_entry[0] == entry[0]:
            return True
    return False

def unique_morphems(morph_result):
    uniqueEntry_list = []
    
    for case in morph_result:
        for entry in case:
            if check_equal(entry, uniqueEntry_list) == False:
                uniqueEntry_list.append(entry)    

    #sorted1 = sorted(uniqueEntry_list, key=lambda k: k[1])
    sorted2 = sorted(uniqueEntry_list, key=lambda k: k[2])
    return sorted2

In [14]:
def prepare_for_viterbi(sent):
    final_entry_list = []
    str_idx = 0
    space_idx = []
        
    for i, token in enumerate(sent.split()):
        
        morph_result = morph.extract(token)

        final_entry_list += unique_morphems(orgin_morph_idx(morph_result, str_idx))
        str_idx = unique_morphems(orgin_morph_idx(morph_result, str_idx))[-1][2]+1
        
        if not i==len(sent.split())-1:
            space_idx.append(str_idx)
        
    return final_entry_list, space_idx

In [15]:
sent = '우선 신고전주의정신을 본받자'

In [16]:
prepare_for_viterbi(sent)[0]

[['우', 0, 1],
 ['서', 2, 3],
 ['ㄴ', 4, 4],
 ['선', 2, 4],
 ['우선', 0, 4],
 ['신', 5, 7],
 ['고', 8, 9],
 ['신고', 5, 9],
 ['전', 10, 12],
 ['고전', 8, 12],
 ['주의', 13, 16],
 ['고전주의', 8, 16],
 ['정', 17, 19],
 ['정신', 17, 22],
 ['을', 23, 25],
 ['본', 26, 28],
 ['받', 29, 31],
 ['본받', 26, 31],
 ['자', 32, 33]]

In [17]:
final_collect_entry_anly = prepare_for_viterbi(sent)[0]
len_target_word = prepare_for_viterbi(sent)[0][-1][-1]

def recursive_merging(collection, index_set, path, list_global_cases):
    temp_path = path[:]
    for index in index_set:
        path.append(index)
        index_set = [e for e in collection if int(e[1]) == index[2]+1] # first position
        if len(index_set) == 0: # the end point
            # save to global and then game over
            list_global_cases.append(path)
        else:
            # recursive keep going
            recursive_merging(collection, index_set, path, list_global_cases) # recursive function
        path = temp_path[:] # recursive돌아서 망가진 path 원래대로 복귀

In [18]:
# sorting based on start index
final_collect_entry_anly = sorted(final_collect_entry_anly, key = lambda x: int(x[1]))

# 오류확인 1. 처음과 끝의 경계는 아래와 같이 체크하면 된다. (중간에 있는 오류는 다음단계에서 확인한다.)
assert(final_collect_entry_anly[0][1] == 0) # 처음이 index 0으로 시작하지 않을 경우.
assert(final_collect_entry_anly[-1][2] == len_target_word) # 마지막이 word길이만큼의 index가 아닐 경우.

### merging / combining all cases using recursive function
list_global_cases, path = [], []
# filtering only index0 (Starting point to recursive function)
index0_entry_anly_set = [ele for ele in final_collect_entry_anly if int(ele[1]) == 0] # first position
# recursive merging
recursive_merging(final_collect_entry_anly, index0_entry_anly_set, path, list_global_cases)


# 오류확인 2. 처음부터 끝까지 이어지는 하나의 case도 없을 경우.
fullpath_check = False
for i in range(0, len(list_global_cases)):
    if list_global_cases[i][-1][2] == len_target_word:
        fullpath_check = True
assert(fullpath_check == True) # 오류 난다면 사전 lookup 문제이다

In [19]:
### Sequence가 0에서부터 len_target_word-1까지 이어지지 않으면 삭제한다.
for i, seq in enumerate(list_global_cases):
    if seq[0][1] != 0 or seq[-1][2] != len_target_word:
        list_global_cases[i] = 'to-be-deleted'
# remove elements, which has 'to-be-deleted' in the first position        
list_global_cases = [elem for elem in list_global_cases if elem != 'to-be-deleted']

In [20]:
list_global_cases

[[['우', 0, 1],
  ['서', 2, 3],
  ['ㄴ', 4, 4],
  ['신', 5, 7],
  ['고', 8, 9],
  ['전', 10, 12],
  ['주의', 13, 16],
  ['정신', 17, 22],
  ['을', 23, 25],
  ['본', 26, 28],
  ['받', 29, 31],
  ['자', 32, 33]],
 [['우', 0, 1],
  ['서', 2, 3],
  ['ㄴ', 4, 4],
  ['신', 5, 7],
  ['고', 8, 9],
  ['전', 10, 12],
  ['주의', 13, 16],
  ['정신', 17, 22],
  ['을', 23, 25],
  ['본받', 26, 31],
  ['자', 32, 33]],
 [['우', 0, 1],
  ['서', 2, 3],
  ['ㄴ', 4, 4],
  ['신', 5, 7],
  ['고전', 8, 12],
  ['주의', 13, 16],
  ['정신', 17, 22],
  ['을', 23, 25],
  ['본', 26, 28],
  ['받', 29, 31],
  ['자', 32, 33]],
 [['우', 0, 1],
  ['서', 2, 3],
  ['ㄴ', 4, 4],
  ['신', 5, 7],
  ['고전', 8, 12],
  ['주의', 13, 16],
  ['정신', 17, 22],
  ['을', 23, 25],
  ['본받', 26, 31],
  ['자', 32, 33]],
 [['우', 0, 1],
  ['서', 2, 3],
  ['ㄴ', 4, 4],
  ['신', 5, 7],
  ['고전주의', 8, 16],
  ['정신', 17, 22],
  ['을', 23, 25],
  ['본', 26, 28],
  ['받', 29, 31],
  ['자', 32, 33]],
 [['우', 0, 1],
  ['서', 2, 3],
  ['ㄴ', 4, 4],
  ['신', 5, 7],
  ['고전주의', 8, 16],
  ['정신', 17, 22],
  ['을', 23,

In [21]:
[case for case in prepare_for_viterbi(sent)[0] if case[1] <= 4 and case[2] <= 4]

[['우', 0, 1], ['서', 2, 3], ['ㄴ', 4, 4], ['선', 2, 4], ['우선', 0, 4]]

## Viterbi Alogorithm

In [22]:
def idxPair_tuple(list):
    new_list = [0]*len(list)
    str_idx = 0
    for i, num in enumerate(list):
        new_list[i] = (str_idx, num)
        str_idx = num+1
    return new_list

In [23]:
def viterbi(input):

    entry_list = input[0]
    end_idx = entry_list[-1][2]
    space_idx = input[1]

    ###
    ### 2개 이상인 end index list (1개의 형태소를 결정해야되는 지점)
    only_endIdx_list = [case[2] for case in entry_list]
    many_endIdx_list = [item for item, count in collections.Counter(only_endIdx_list).items() if count > 1]
    # many_endIdx_list 의미 
    # ex. [4, 9, 12, 16, 31]이면 이 지점에서 하나의 형태소를 결정지어야 한다. (한국어 특성때문)
    # 이 지점에서 결정짓지 않으면 viterbi라 하더라도 계산량이 매우 많아진다.
    many_idxPair_list = idxPair_tuple(many_endIdx_list)
    print(many_idxPair_list)
#     ###
#     ### start index 기준 엔트리 나열
#     startIdx_entry_list = []
#     for i in range(0, end_idx+1):
#         entries = [ent for ent in prepare_for_viterbi(sent)[0] if ent[1] == i]
#         if len(entries) != 0:
#             startIdx_entry_list.append(entries)
#     print(startIdx_entry_list)


    for start, end in many_idxPair_list:
        
        # endIdx 이하 entry들 filtering
        print([case for case in entry_list if case[1] >= start and case[2] <= end])
        
        break

In [24]:
viterbi(prepare_for_viterbi(sent))

[(0, 4), (5, 9), (10, 12), (13, 16), (17, 31)]
[['우', 0, 1], ['서', 2, 3], ['ㄴ', 4, 4], ['선', 2, 4], ['우선', 0, 4]]


In [25]:
viterbi(prepare_for_viterbi(sent))

[(0, 4), (5, 9), (10, 12), (13, 16), (17, 31)]
[['우', 0, 1], ['서', 2, 3], ['ㄴ', 4, 4], ['선', 2, 4], ['우선', 0, 4]]


In [26]:
morph_result = list_global_cases

In [27]:
# check all words in 
# 그냥 형태소 분석의 출력 경우의 수 각각 확률을 구해주고 비교하는 거라면... 영어처럼 viterbi를 수행해도 된다.

In [28]:
def extract_maxProb(tag_list, total_probState, i, j, prob_pTagcTag):
    
    cur_tag = tag_list[j]
    temp = [0] * len(tag_list)
    
    for k, _ in enumerate(tag_list):
        temp[k] = prob_pTagcTag[(tag_list[k], cur_tag)] * total_probState[i-1][k]
    
    argmax_idx = np.where(np.array(temp) == np.array(temp).max())
    index = argmax_idx[0][0] # (array([3], dtype=int64),) -> [0][0]해줘야 index 3가 추출된다.
    max_prob = np.array(temp).max()
    
    return max_prob, index

In [29]:
all_pos_cases = []
all_jointprob_cases = []
prob_case = [0] * len(morph_result)

tag_list = list(freq_Tag.keys()) # 주의: tag_list와 freq_Tag.keys()의 배열 순서는 다르다.

for case in morph_result: # 형태소 분석 조합 경우의 수
    
    output_probState = [[0] * len(tag_list)] * len(case) 
    total_probState = [[0] * len(tag_list)] * len(case) # total probability on the state
    total_idxState = [[-1] * len(tag_list)] * len(case) # optimal path (=[t-1] index) )
    pos_case = ['n'] * len(case)
    joint_prob = 0
    
    ### Step 1: Assigning output probabliities
    for i, morph in enumerate(case): # 형태소 경우의 수
        cur_syllable = morph[0]
        for j, tag in enumerate(tag_list): # POS Tag 경우의 수
            output_probState[i][j] = prob_TagWord[(tag, cur_syllable)]
    
    ### Step 2: Storing only maximum probabiliteis to each state by multiplying transition probabilities
    for i, morph in enumerate(case): # 형태소 경우의 수
        for j, tag in enumerate(tag_list): # POS Tag 경우의 수       
            
            cur_tag = tag_list[j]
            cur_probState = total_probState[i][j]
            
            if i==0: # prev time에 START밖에 없으므로 max확률 찾을필요없이 그냥 다 저장한다.
                total_probState[0][j] = prob_pTagcTag[('START', tag)] * output_probState[0][j]
                # i==0일 때는 total_idxState[0][j]에 idx를 저장할 필요가 없다. 왜냐면, 어차피 무조건 START와 연결되기 때문이다.
            else:
                total_probState[i][j], total_idxState[i][j] = extract_maxProb(tag_list, total_probState, i, j, prob_pTagcTag)
    
    ### Step 3: Storing argmax index for connecting previous time
    temp = [0] * len(tag_list)
    for j, _ in enumerate(total_probState[-1]):
        temp[j] = total_probState[-1][j] * prob_pTagcTag[(tag_list[j], 'END')]
    
    # only for END tag
    end_idxState = np.where(np.array(temp) == np.array(temp).max())
    end_idxState = end_idxState[0][0] # for extracting only index
    
    ### Step 4: Doing Back-tracking
    tag_idx = -1
    for i, x in enumerate(reversed(pos_case)):
        idx = (len(pos_case)-1) - i
        
        if i==0: # last syllable
            pos_case[idx] = tag_list[end_idxState]
            joint_prob = total_probState[idx][end_idxState]
            tag_idx = end_idxState # for sending next time
        else:
            pos_case[idx] = tag_list[total_idxState[idx][tag_idx]]
            joint_prob *= total_probState[idx][tag_idx]
            
            tag_idx = total_idxState[idx][tag_idx] # for sending next time
            
    
    
    
    all_pos_cases.append(pos_case)
    all_jointprob_cases.append(joint_prob)
    
    #break
    

NameError: name 'prob_TagWord' is not defined

In [ ]:
all_pos_cases

In [ ]:
all_jointprob_cases

In [ ]:
all_jointprob_cases = np.array(all_jointprob_cases)
indices = np.where(all_jointprob_cases == all_jointprob_cases.max())

In [ ]:
answer = np.where(all_jointprob_cases == all_jointprob_cases.max())
answer_idx = answer[0][0] # 2개 이상이더라도 그냥 맨 처음에 있는 것을 argmax로 하자.

## Answer

In [ ]:
morph_result[answer_idx]

In [ ]:
all_pos_cases[answer_idx]

## Appendix

In [ ]:
# # dictionary filtering
# cut_threshold = 30
# freq_allTag = dict((k, v) for k, v in freq_allTag.items() if v > cut_threshold)

In [ ]:
# tag 빈도수
# tag bigram 빈도수
# word, tag 쌍 빈도수

In [ ]:
# 전이확률
# bigram확률